# Wipeout
On November 1st 2019 I went out for a bike ride and wiped out. Not a little loss of control or a little skid, we're talking flying off the bike in traffic kind of wipe out. I finished my ride as I had planned, and when I was telling my wife about the incident she asked me where but I couldn't really remember. I did remember a few things though - can you help me track down where I wiped out from my ride log data, which I put in `wipeout.csv`?

Specifically I remember that:
1. I launched off the bike. So at some point both my reported speed `enhanced_speed` and my reported  `cadence` from my wearables would have been 0.
2. I was at least 5 minutes into my ride at the time, headed out of the city.
4. I do remember stopping during my ride *before* I wiped out to give an in promptu lecture to a motorist on Miller as to whether cars belonged in the bike lane.

Where and when did I wipe out (in units of `timestamp`, `position_lat_degrees` and `position_long_degrees`).

In [2]:
import pandas as pd
df = pd.read_csv("wipeout.csv")
df.head()

,timestamp,cadence,distance,enhanced_altitude,enhanced_speed,fractional_cadence,heart_rate,position_lat,position_long,speed,timestamp.1,position_lat_degrees,position_long_degrees
0,2019-11-01 20:12:03,NaN,0.00,281.6,0.000,NaN,84.0,504424439.0,-999065791.0,NaN,2019-11-01 20:12:03,42.280368,-83.740727
1,2019-11-01 20:12:04,NaN,0.00,281.6,0.000,NaN,84.0,504424439.0,-999065791.0,NaN,2019-11-01 20:12:04,42.280368,-83.740727
2,2019-11-01 20:12:07,NaN,0.00,281.6,0.000,NaN,87.0,504424439.0,-999065791.0,NaN,2019-11-01 20:12:07,42.280368,-83.740727
3,2019-11-01 20:12:08,NaN,0.32,281.6,1.036,NaN,88.0,504424447.0,-999065835.0,NaN,2019-11-01 20:12:08,42.280369,-83.740731
4,2019-11-01 20:12:10,NaN,3.30,281.8,2.976,NaN,89.0,504424476.0,-999066382.0,NaN,2019-11-01 20:12:10,42.280371,-83.740777


In [4]:
df.timestamp.min(), df.timestamp.max()

('2019-11-01 20:12:03', '2019-11-01 21:25:05')

In [22]:
#Considering Chris' criteria let's look at stops after the first five minutes based on enhanced speed and cadence == 0
df[(df['timestamp']>'2019-11-01 20:17')&(df['enhanced_speed'] == 0)&(df['cadence'] == 0)]

,timestamp,cadence,distance,enhanced_altitude,enhanced_speed,fractional_cadence,heart_rate,position_lat,position_long,speed,timestamp.1,position_lat_degrees,position_long_degrees,position_lat_degrees_mercantor
143,2019-11-01 20:18:31,0.0,1811.68,252.2,0.0,0.0,137.0,504493081.0,-999276526.0,NaN,2019-11-01 20:18:31,42.286122,-83.758391,46.747751
144,2019-11-01 20:18:34,0.0,1811.68,252.0,0.0,0.0,134.0,504493081.0,-999276526.0,NaN,2019-11-01 20:18:34,42.286122,-83.758391,46.747751
145,2019-11-01 20:18:39,0.0,1811.68,251.8,0.0,0.0,131.0,504493081.0,-999276526.0,NaN,2019-11-01 20:18:39,42.286122,-83.758391,46.747751
146,2019-11-01 20:18:42,0.0,1811.68,251.8,0.0,0.0,128.0,504493081.0,-999276526.0,NaN,2019-11-01 20:18:42,42.286122,-83.758391,46.747751
147,2019-11-01 20:18:44,0.0,1811.68,251.8,0.0,0.0,125.0,504493081.0,-999276526.0,NaN,2019-11-01 20:18:44,42.286122,-83.758391,46.747751
148,2019-11-01 20:18:47,0.0,1811.68,251.8,0.0,0.0,122.0,504493081.0,-999276526.0,NaN,2019-11-01 20:18:47,42.286122,-83.758391,46.747751
149,2019-11-01 20:18:52,0.0,1811.68,251.8,0.0,0.0,118.0,504493097.0,-999276555.0,NaN,2019-11-01 20:18:52,42.286123,-83.758393,46.747753
226,2019-11-01 20:24:40,0.0,3797.54,280.4,0.0,0.0,127.0,504592053.0,-999531545.0,NaN,2019-11-01 20:24:40,42.294417,-83.779766,46.758966
255,2019-11-01 20:26:26,0.0,4485.11,290.0,0.0,0.0,144.0,504661370.0,-999544061.0,NaN,2019-11-01 20:26:26,42.300227,-83.780815,46.766821


In [40]:
#Let's visualize the data to help us identify the time spent at each stop. 

import folium
#Transform positions to degrees for mapping
df['position_lat_degrees'] = df["position_lat"] * ( 180 / 2**31 )
df["position_long_degrees"] = df["position_long"] * ( 180 / 2**31 )

import math
def lat2y(a):
    return 180.0/math.pi*math.log(math.tan(math.pi/4.0+a*(math.pi/180.0)/2.0))

df["position_lat_degrees_mercantor"]=df["position_lat_degrees"].apply(lat2y)
df = df.dropna(subset=['position_lat_degrees', 'position_long_degrees'])


m=folium.Map(location=[42.28,-83.75], zoom_start=13)
#plotting the bike route
route = folium.PolyLine(locations=zip(df["position_lat_degrees"],df["position_long_degrees"]), 
                            weight=2, color = "#003f5c").add_to(m)

#plotting the stops based on our criteria. Color will be bolder if the stop was longer.
for stop in df[(df['timestamp']>'2019-11-01 20:17')&(df['enhanced_speed'] == 0)&(df['cadence'] == 0)].index:
    folium.Circle(radius=50, location=[df["position_lat_degrees"].iloc[stop],df["position_long_degrees"].iloc[stop]],
                  popup='Stop', color='red', opacity = 0.5).add_to(m)
    
display(m)

In [ ]:
#Seems the first stops was longer than the other two - could have been when Chris was arguing with the biker? 
#There are two stops on Miller Avenue and this is the first one. 

In [36]:
#Looking at the second and third stop, we can see that the second stop was followed by a few records of slow cadence 
#and 0 enhanced speed, which could mean Chris took 8 seconds once falled to get-up, get his bike and ride again. 
#It is fast but not impossible if traffic was heavy as he likely did not want to stop traffic on the bike lane. 
#The third stop on the other end seems to indicate the stop was more brief (4 seconds max), which makes it a less good 
#candidate for a fall, and a better candidate for a stop. Looking at the map was can see it happened close to a roundabout, 
#could have simply be a stop sign or giving priority to cars.
second_stop = df[(df['timestamp']>'2019-11-01 20:24:30')&(df['timestamp']<'2019-11-01 20:24:55')]
third_stop = df[(df['timestamp']>'2019-11-01 20:26:20')&(df['timestamp']<'2019-11-01 20:26:50')]
second_stop
#third_stop

,timestamp,cadence,distance,enhanced_altitude,enhanced_speed,fractional_cadence,heart_rate,position_lat,position_long,speed,timestamp.1,position_lat_degrees,position_long_degrees,position_lat_degrees_mercantor
224,2019-11-01 20:24:34,69.0,3790.03,280.0,5.067,0.0,128.0,504591218.0,-999529571.0,NaN,2019-11-01 20:24:34,42.294347,-83.779601,46.758871
225,2019-11-01 20:24:37,69.0,3797.54,280.0,1.633,0.0,127.0,504591989.0,-999531403.0,NaN,2019-11-01 20:24:37,42.294412,-83.779754,46.758959
226,2019-11-01 20:24:40,0.0,3797.54,280.4,0.000,0.0,127.0,504592053.0,-999531545.0,NaN,2019-11-01 20:24:40,42.294417,-83.779766,46.758966
227,2019-11-01 20:24:45,5.0,3797.54,280.4,0.000,0.0,130.0,504592053.0,-999531545.0,NaN,2019-11-01 20:24:45,42.294417,-83.779766,46.758966
228,2019-11-01 20:24:47,5.0,3797.54,280.6,0.000,0.0,133.0,504592060.0,-999531723.0,NaN,2019-11-01 20:24:47,42.294418,-83.779781,46.758967
229,2019-11-01 20:24:48,5.0,3797.92,280.6,0.383,0.0,134.0,504592070.0,-999531900.0,NaN,2019-11-01 20:24:48,42.294419,-83.779796,46.758968
230,2019-11-01 20:24:50,13.0,3805.33,280.6,3.919,0.0,134.0,504592227.0,-999532551.0,NaN,2019-11-01 20:24:50,42.294432,-83.779851,46.758986
231,2019-11-01 20:24:53,57.0,3820.07,280.6,5.412,0.0,131.0,504592908.0,-999533924.0,NaN,2019-11-01 20:24:53,42.294489,-83.779966,46.759063


In [38]:
#In conclusion, I think Chris fell on Miller Avenue close to Allen Creek Preschool, before turning on North Maple Road, at 8.24pm.
fall = df[['timestamp', 'position_lat_degrees', 'position_long_degrees']].iloc[226]
fall

timestamp                2019-11-01 20:24:40
position_lat_degrees                 42.2944
position_long_degrees               -83.7798
Name: 226, dtype: object

In [41]:
m=folium.Map(location=[42.28,-83.75], zoom_start=13)
#plotting the bike route
route = folium.PolyLine(locations=zip(df["position_lat_degrees"],df["position_long_degrees"]), 
                            weight=2, color = "#003f5c").add_to(m)

#plotting the fall
folium.Circle(radius=100, location=[df['position_lat_degrees'].iloc[226], df['position_long_degrees'].iloc[226]], 
              popup='Fall', color='blue').add_to(m)

display(m)